In [5]:
import numpy as np
import joblib
import tkinter as tk
from tkinter import filedialog
import os 
import pandas as pd
def BMI(h, w):
    if h == 0:
        return None
    return round(w/(h/100)**2,3)
def discretization(feature, bins):
    # age: [20,40,65] 
    # BMI: [0,18.5,24,27,30,35]
    for i,v in enumerate(bins):
        if feature<v:
            return i
    return len(bins)
def price_range(pred, bins = [15000,20000, 24000, 29000, 35000, 40000]):
    for i in range(len(bins)-1):
        if pred==i:
            return (bins[i],bins[i+1])
#%%

class medInsuranceData():
    def __init__(self,file_path):
        data={}
        file_path = '/'.join(file_path.split('\\'))
        if os.path.isdir(file_path):
            file_names = os.listdir(file_path)
            file_dir = file_path+'/'
        elif os.path.isfile(file_path):
            file_names = [file_path.split('/')[-1]]
            file_dir = file_path[:file_path.index(*file_names)]
        #2.1 fixed problem of data selection not available
        elif len(file_path.split(','))>1:
            file_names = [v.split('/')[-1] for v in file_path.split(',')] 
            file_dir = file_path[:file_path.index(file_names[0])]
        else:
            raise FileNotFoundError('File not Found!')
        found_file = False
        for file_name in file_names:
            if file_name.split('.')[-1]=='csv':
                found_file = True
                raw_data = pd.read_csv(file_dir+file_name)
                data[file_name]=self.preprocessing(raw_data)
            if not found_file:
                raise FileNotFoundError('File not Found!')
        self.files = file_names       
        self.file_path = file_path
        self.data = data
    def preprocessing(self,df):
        df["BMI"] = df.Weight/(df.Height/100)**2
        df["BMI"] = pd.to_numeric(pd.cut(df['BMI'], bins=[0,18.5,24,27,30,35,1000], labels=[0,1,2,3,4,5]))
        df["Age_new"]=pd.to_numeric(pd.cut(df["Age"],bins=[0,20,40,65,100], labels = [0,1,2,3]))
        return df
    def predict(self, model):
        os.makedirs('result',exist_ok=True)
        for file in self.files:
            x = self.data.get(file)
            pred = model.predict(x.drop(columns = ["PremiumPrice"]))
            
            pred_range = []
            for _ in pred:
                pred_range.append(price_range(_))
            
            pd.DataFrame(pred_range).to_csv(f'result/Prediction_{file}')
    
    #%%
class predictionApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)    
        self.title('demo') 
        self.model = joblib.load('result/random_forest.joblib')
        self._checkbox = ['Diabetes','BloodPressureProblems','AnyTransplants',
                          'AnyChronicDiseases', 'KnownAllergies', 'HistoryOfCancerInFamily']
        self._entry = ['Age','Height', 'Weight','NumberOfMajorSurgeries']
        self.features = ['Age', 'Diabetes', 'BloodPressureProblems', 'AnyTransplants',
                         'AnyChronicDiseases', 'Height', 'Weight', 'KnownAllergies',
                         'HistoryOfCancerInFamily', 'NumberOfMajorSurgeries']
        self.features_ = ['BMI','age_new']
         
        self.switch_page(None,self.start_page)
    def switch_page(self,f,next_page):
        if f is not None:
            f.destroy()
        next_page()
        
    def start_page(self):
        f = tk.Frame(self, width=330, height=100, borderwidth=15)	
        f.grid_propagate(0)
        f.grid(row=0, column=0)
        label_mode = tk.Label(f, text = 'Choose predict mode')
        label_mode.grid(row=0, column=1,sticky='we')
        button_1 = tk.Button(f, text = 'Personal Test', 
                                command = lambda: self.switch_page(f,self.input_page))
        button_1.grid(row=1,column=0,pady=10)
        button_2 = tk.Button(f, text = 'Test on Data', 
                                command = lambda: self.switch_page(f,self.data_page))
        button_2.grid(row=1,column=2,pady=10)
        
    def predict(self, user_data):
        age_ = discretization(user_data['Age'].get(), [20,40,65,100])
        BMI_ = discretization(user_data['BMI'].get(), [18.5,24,27,30,35])
        x = np.array([user_data[_].get() for _ in self.features]+[age_,BMI_]).reshape(1,-1)
        pred = self.model.predict(x)
        return pred
   
    def input_page(self):
        def update_BMI(*args):
            h,w = var['Height'].get(),var['Weight'].get()
            if type(h)==float and type(h)==float:
                var['BMI'].set(BMI(h,w))
        
        f = tk.Frame(root, width=500, height=300, borderwidth=15)
        f.grid_propagate(0)
        f.grid(row=0, column=0)
        entry,check, label, var = {},{},{},{}
        for i, feature in enumerate(self._entry):
            if feature in ['Age','NumberOfMajorSurgeries']:
                var[feature] = tk.IntVar()
            else:
                var[feature] = tk.DoubleVar()
            label[feature] = tk.Label(f, text = feature)
            label[feature].grid(row=i,column=0,sticky = 'w')
            entry[feature] = tk.Entry(f,textvariable = var[feature])
            entry[feature] = tk.Entry(f,textvariable = var[feature])
            entry[feature].grid(row=i, column=1, columnspan = 2,sticky = 'we')
        
        for i, feature in enumerate(self._checkbox,len(self._entry)):
            var[feature] = tk.IntVar()
            label[feature] = tk.Label(f, text = feature)
            label[feature].grid(row=i,column=0,sticky = 'w')
            check[feature] = tk.Checkbutton(f, text = '', variable = var[feature],
                                            onvalue = 1, offvalue = 0, height=1)
            check[feature].grid(row=i, column=1,sticky = 'w')
        var['BMI'] = tk.DoubleVar()
        for feature in ['Height','Weight']:
            var[feature].trace('w',update_BMI)
        BMI_label = tk.Label(f,text =  'BMI = ')
        BMI_label.grid(row=1,column=4,sticky = 'w')
        BMI_label_ = tk.Label(f,textvariable =  var['BMI'])
        BMI_label_.grid(row=2,column=4,sticky = 'w')
        ## prediciton        
        var_pred_txt = tk.StringVar()
        var_pred_txt.set('Prediction:')
        results_label = tk.Label(f,textvariable =  var_pred_txt)
        results_label.grid(row=6,column=4)
        button_pred = tk.Button(f, text = 'Predict', 
                                command = lambda: var_pred_txt.set(f'Prediction: {price_range(self.predict(var)[0])}'))
        button_pred.grid(row=16,column=4,pady=10,padx=10)
        
        button_return = tk.Button(f, text = 'return', 
                                  command = lambda: self.switch_page(f,self.start_page))
        button_return.grid(row=16,column=0,pady=10,padx=10)
        
    def data_page(self):
        def predict():
            var_done_pred.set('......')
            data = medInsuranceData(path.get())
            data.predict(self.model)
            var_done_pred.set('Done')
            
        f = tk.Frame(root, width=500, height=100, borderwidth=15)	
        f.grid_propagate(0)
        f.grid(row=0, column=0)
        
        def selectPath(): 
            path_tuple = filedialog.askopenfilenames(initialdir=path.get(), filetypes = (("csv file","*.csv"),))
            path_ = (',').join(path_tuple)
            path.set(path_)
            if path_ == '':
                path.set(os.getcwd())
        path = tk.StringVar()
        path.set(os.getcwd())
        path_label = tk.Label(f,text = "target file:")

        path_entry = tk.Entry(f, textvariable = path)

        path_button = tk.Button(f, text = "select", command = selectPath)
        path_label.grid(row = 0, column = 0, padx=(0,0))
        path_entry.grid(row = 0, column = 2, columnspan = 4, ipadx=100, padx=5, sticky='we')
        path_button.grid(row = 0, column = 6)
        button_pred = tk.Button(f, text = 'Predict', 
                                command = predict)
        button_pred.grid(row=16,column=6,pady=20)
        
        var_done_pred = tk.StringVar()        
        label_done = tk.Label(textvariable = var_done_pred)
        label_done.grid(row=17,column=6)
        
        button_return = tk.Button(f, text = 'return', 
                                  command = lambda: self.switch_page(f,self.start_page))
        button_return.grid(row=16,column=0,pady=10)
        
root = predictionApp()

root.mainloop()


    
# label_1 = tk.Label(window, text='Hello World', bg='yellow', fg='#263238', font=('Arial', 12))
# label_1.grid(column=0, row=0)



Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ess\anaconda3\envs\fintech_final\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\ess\AppData\Local\Temp/ipykernel_20416/2931899963.py", line 110, in update_BMI
    h,w = var['Height'].get(),var['Weight'].get()
  File "C:\Users\ess\anaconda3\envs\fintech_final\lib\tkinter\__init__.py", line 529, in get
    return self._tk.getdouble(self._tk.globalgetvar(self._name))
_tkinter.TclError: expected floating-point number but got ""
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ess\anaconda3\envs\fintech_final\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\ess\AppData\Local\Temp/ipykernel_20416/2931899963.py", line 110, in update_BMI
    h,w = var['Height'].get(),var['Weight'].get()
  File "C:\Users\ess\anaconda3\envs\fintech_final\lib\tkinter\__init__.py", line 529, in get
    return s